# overview

In [1]:
# default package
import logging
import sys
import os 
import pathlib

In [22]:
# third party package
import matplotlib.pyplot as plt
import seaborn as sns
import wandb 
import pandas as pd 
import torch
import torch.nn as nn

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
# graph settings
sns.set()

In [7]:
def cd_project_root_()->None:
    current=pathlib.Path().cwd()
    if current.stem=="notebooks":
        os.chdir(current.parent)
    logger.info(f"current directory: {pathlib.Path().cwd()}")

cd_project_root_()

INFO:__main__:current directory: /workspaces/load_to_goal/GitHub/kaggle-past-comp/atma-11


## oof

In [9]:
run = wandb.init()
artifact = run.use_artifact('tmdiesel/atma11/dataset-oof-gueerqe6:v0', type='dataset')
artifact_dir = artifact.download()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tmdiesel (use `wandb login --relogin` to force relogin)


In [12]:
df=pd.read_csv(pathlib.Path(artifact_dir)/"oof.csv")

In [18]:
df_train=pd.read_csv("data/raw/train.csv")

In [25]:
df_train["target"]
loss = nn.MSELoss()
input = torch.tensor(df["pred"])
target = torch.tensor(df_train["target"])
output = loss(input, target)
output

tensor(0.7348, dtype=torch.float64)

## test

In [29]:
run_id_list=["29jy3nf0","2nppyvgr","3iekyqc3","c7nq6h9a","gueerqe6"]
df_list=[]

for fold,run_id in enumerate(run_id_list):
    run = wandb.init()
    artifact = run.use_artifact(f'tmdiesel/atma11/dataset-sub-{fold}-{run_id}:v0', type='dataset')
    artifact_dir = artifact.download()
    df_list.append(pd.read_csv(pathlib.Path(artifact_dir)/f"sub_{fold}.csv"))

In [44]:
for i in range(5):
    df_list[i].loc[df_list[i]["target"]>3,"target"]=3

In [48]:
df_sum=df_list[0]["target"]+df_list[1]["target"]+df_list[2]["target"]+df_list[3]["target"]+df_list[4]["target"]
df_mean=df_sum/5

In [52]:
df_mean.to_csv("data/processed/sub/sub_1.csv",index=False)

## check